# Cellucid Jupyter Hooks + Sessions (HE developmental example)

This notebook is a **hands-on test** for the Jupyter integration:
- embedding + connectivity
- hooks (`on_ready`, `on_selection`, `on_hover`, `on_click`)
- `viewer.state` snapshots
- no-download session capture (`viewer.get_session_bundle`)
- apply session → `AnnData`

Dataset: `cellucid-python/data/experiments/he_developmental_complete_with_3d_umap.h5ad`


In [1]:
from pathlib import Path
import sys

HERE = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()

def find_project_root(start: Path) -> Path:
    """Locate the `cellucid-python` repo root (folder containing `pyproject.toml`)."""
    for candidate in [start, *start.parents]:
        if (candidate / "pyproject.toml").exists():
            return candidate
        if (candidate / "cellucid-python" / "pyproject.toml").exists():
            return candidate / "cellucid-python"
    return start

PROJECT_ROOT = find_project_root(HERE)
SRC_DIR = PROJECT_ROOT / "src"
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))


In [2]:
from __future__ import annotations

from pathlib import Path

import anndata

from cellucid import show_anndata


In [3]:
# Locate the dataset (works whether your CWD is the repo root or this examples folder).
candidates = [
    Path('data/experiments/he_developmental_complete_with_3d_umap.h5ad'),
    Path('../../../../data/experiments/he_developmental_complete_with_3d_umap.h5ad'),
]

DATASET = next((p for p in candidates if p.exists()), None)
if DATASET is None:
    raise FileNotFoundError('Could not find the HE developmental dataset in expected locations')

DATASET = DATASET.resolve()
DATASET


PosixPath('/Users/kemalinecik/git_nosync/_/cellucid-python/data/experiments/he_developmental_complete_with_3d_umap.h5ad')

In [4]:
# Load AnnData in backed mode (keeps memory usage lower).
adata = anndata.read_h5ad(DATASET, backed='r')
adata


AnnData object with n_obs × n_vars = 71650 × 8192 backed at '/Users/kemalinecik/git_nosync/_/cellucid-python/data/experiments/he_developmental_complete_with_3d_umap.h5ad'
    obs: 'sample_ID', 'organ', 'age', 'cell_type', 'sex', 'sex_inferred', 'concatenated_integration_covariates', 'integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse', 'n_genes', 'LVL3', 'LVL2', 'LVL1', 'LVL0', '_scvi_batch', '_scvi_labels'
    uns: 'metrics', 'neighbors', 'rank_genes_groups', 'umap'
    obsm: 'Unintegrated', 'X_pca', 'X_umap', 'X_umap_1d', 'X_umap_2d', 'X_umap_3d', 'harmony', 'scvi'
    obsp: 'connectivities', 'distances'

In [5]:
# Launch the viewer.
# In notebooks, this auto-displays an iframe and starts a background server.
viewer = show_anndata(DATASET, height=650)
viewer


AnnDataViewer(port=8766, status=running)

In [6]:
# Connectivity diagnostics (useful in VSCode/JupyterLab where devtools can be limited).
conn = viewer.debug_connection()
conn

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

{'viewer_id': '8946d266c3167f62',
 'viewer_url': 'http://127.0.0.1:8766/?jupyter=true&viewerId=8946d266c3167f62&viewerToken=ff202dd528bb05b2c96221170d15f3ab&anndata=true',
 'server_url': 'http://127.0.0.1:8766',
 'displayed': True,
 'notebook_context': {'in_jupyter': True,
  'notebook_type': 'vscode',
  'kernel_id': '8a992090-8d63f1dc270fe8b8684a7c14',
  'can_iframe': True,
  'preferred_display': 'iframe'},
 'server_running': True,
 'web_ui': {'proxy_cache_dir': '/var/folders/y7/7c17s0l57szdjc1cdc9dmnpm0000gn/T/cellucid-web-cache',
  'proxy_source': 'https://www.cellucid.com',
  'cache': {'cache_dir_exists': True,
   'index_html_exists': True,
   'index_html_bytes': 77839,
   'index_html_build_id': '2025-12-31T23:59:59Z-theislab-datasets',
   'index_html_mtime': 1767228114.2268932}},
 'state': {'ready': {'n_cells': 71650, 'dimensions': 3},
  'last_event_type': 'hover',
  'last_updated_at': 118244.409718125},
 'client_server_url': 'http://127.0.0.1:8766',
 'server_health': {'status': 'o

In [ ]:
# Hook examples (UI → Python).
@viewer.on_ready
def _on_ready(event):
    print('READY:', event)

@viewer.on_click
def _on_click(event):
    print('CLICK:', event.get('cell'), 'button=', event.get('button'))

@viewer.on_hover
def _on_hover(event):
    # Hover is frequent; only print when entering a cell.
    cell = event.get('cell')
    if cell is not None:
        print('HOVER:', cell)

@viewer.on_selection
def _on_selection(event):
    # Note: in notebooks, selection is emitted when you CONFIRM a selection into a highlight group.
    cells = event.get('cells') or []
    print('SELECTION:', len(cells), 'source=', event.get('source'))


## Try it

1. Wait for the viewer to finish loading.
2. Make a selection in the UI and **confirm** it into a highlight group.
3. Come back and run the next cells to pull state and capture a session bundle.


In [ ]:
viewer.wait_for_ready(timeout=120)
viewer.state


In [ ]:
# Capture the current session as a Python object (no manual download).
bundle = viewer.get_session_bundle(timeout=120)
len(bundle.list_chunk_ids()), bundle.dataset_fingerprint


In [ ]:
# Apply the session bundle back onto AnnData.
# This currently materializes highlight memberships and categorical user-defined fields.
adata2, summary = bundle.apply_to_anndata(adata, inplace=False, return_summary=True)
summary


In [ ]:
# Inspect the newly added columns (names depend on highlight group IDs and field keys).
[c for c in adata2.obs.columns if c.startswith('cellucid_')]


In [ ]:
# If anything feels off, this report is the fastest way to diagnose connectivity.
viewer.debug_connection()


In [ ]:
# Cleanup: stops the server and freezes the displayed view (non-interactive, visually unchanged).
# viewer.stop()
